In [ ]:
import time, os, sys, multiprocessing
from multiprocessing import Pool
from sqlalchemy import create_engine, MetaData, Table, select

In [ ]:
'''Step 1: Create the vocabulary.txt file'''

def getMimicTexts():
    '''
    Input: N/A
    Output: List[Strings] for all 2 million+ MIMIC texts **lowercase**. 
    We're going to use this List[Strings] to create the set of vocabulary words that is needed for BiLM.
    '''
    t1 = time.time() #start timer
    
    engine = create_engine('sqlite:///mimic.db') #initiated database engine
    conn = engine.connect()
    metadata = MetaData(bind=engine) #init metadata. will be empty
    metadata.reflect(engine) #retrieve db info for metadata (tables, columns, types)
    mydata = Table('mydata', metadata)

    data = []

    #Query db for text. Not efficient. Soz bro.   
    s = select([mydata.c.TEXT]) 
    print(type(s))
    result = conn.execute(s)
    print(type(result))
    for row in result:
        #text
        the_text = row["TEXT"]
        keep_text = the_text.rstrip()
        lower_text = keep_text.lower() #lowercase v important.
        formatted_text = ' '.join(lower_text.split()) #make it all on one line...
        '''TODO: honestly a bit more preprocessing could go here for trailing punctuation. 
        We'll have a problem with 'word.' versus 'word .' but i don't feel like fixing this right now...''' 
        data.append(lower_text)
    
    print(" * Finished step0: done in %0.3fs." % (time.time() - t1))
    return data

In [ ]:
list_of_all_docs = getMimicTexts()

In [ ]:
#### Example ##### 
print("* list_of_all_docs is a: ", type(list_of_all_docs))
print("* documents in list_of_all_docs are: ", type(list_of_all_docs[0]))
print("* Example documents: ", list_of_all_docs[0])

In [ ]:
def getSetOfWords(document):
    '''
    Input: String
    Output: Set(Strings) = This will output the set of tokens in the document
    TODO: What tokenizer do we want to use? I'm just gonna use white space here for now 
    because that's a problem for future-me.
    '''
    words_list: list[string] = document.split()
    unique_words: set = set(words_list)
    return unique_words

unique_words_example = getSetOfWords(list_of_all_docs[0])

In [ ]:
'''Create the vocab.txt that is necessary to run BiLM
'''
def createVocabFile():
    '''
    Input: String = Point it at the text file that contains all of the mimic files!
    '''
    t1 = time.time() #start the timer
    
    pool_size = multiprocessing.cpu_count() #Usin' all yer CPU's my friend. 
    pool = Pool(pool_size)
    print('* created worker pools')
    results0 = pool.map_async(getSetOfWords, list_of_all_docs[0:3])  #TODO: this will be the whole set, not a subset
    print('* initialized map_async to naiveSearchText function with docs')
    print('* did map to getSetOfWords function with docs. WITH async')
    pool.close()
    print('* closed pool')
    pool.join()
    print('* joined pool')
    list_of_sets = [r for r in results0.get() if r is not None] # A BUNCH OF SETS
    print(len(list_of_sets))
    #print(list_of_sets)
    
    #init a set
    giga_set = set()
    #now add all of the sets to it to create one super big set
    quick_and_dirty = [giga_set.update(s) for s in list_of_sets]
    print(len(giga_set))
    print(giga_set)
    
    #we also need to add these AllenNLP specific things
    allen_specific = set(['<S>','</S>','<UNK>'])
    giga_set.update(allen_specific)

    #now output to vocab.txt
    with open("mimic_vocab.txt", "w") as out:
        for word in giga_set:
            out.write(word)
            out.write("\n")
    
    
    print(" * Created mimic_vocab.txt: done in %0.3fs." % (time.time() - t1))


createVocabFile()

In [ ]:
"""We also need the create the mimic_data.txt"""
def createData():
    with open("mimic_data.txt", "w") as out:
        for document in list_of_all_docs[0:3]:
            out.write(document)
            out.write("\n")
createData()

#BOOOO THIS ISN"T WORKING